In [1]:
%run input/Format.ipynb
import ROOT as root
from array import array
root.gErrorIgnoreLevel = root.kFatal
%jsroot on

/home/yoren/.local/lib/python3.10/site-packages/nbformat/__init__.py:96: MissingIDFieldWarning: Cell is missing an id field, this will become a hard error in future nbformat versions. You may want to use `normalize()` on your notebooks before validations (available since nbformat 5.1.4). Previous versions of nbformat are fixing this issue transparently, and will stop doing so in the future.
  validate(nb)


Welcome to JupyROOT 6.30/06


In [2]:
iOption0 = 1
part = 0
N_loc_files = 2

In [3]:
N_centr = 5
N_pt = 1
colors=[1,2,4,root.kGreen+2,root.kMagenta,root.kOrange+4,root.kGray,root.kCyan]
central_bins = [0,20,60,80,20,40,40,60,60,80,80,100]
pt_boarders = [0.5,0.7,1.0,2.0,0,5.0,1.0,5.0,0.5,5]
file_names=["../sim/output/embed/photon_50M_embed_hagedorn_v3_secondhit.root","../sim/output/embed/piminus_10M_embed_hagedorn_v3_secondhit.root",\
            "input/conv_8cut_0728.root","input/my-electron_qa_0728.root"]
part_names = ["#gamma #rightarrow ee","#pi^{-}","conv","data","#gamma #rightarrow ee","#pi^{-}","#pi^{-}","#gamma #rightarrow ee"]
hist_sveto=["veto_sphi_phi_hist_0","veto_sphi_phi_hist_1","veto_sphi_phi_hist_2"]
hist_mass_DCA_names = ["counter_assoc_ghost_hist"]
hists_all = [hist_sveto,hist_mass_DCA_names]
N_type = len(hists_all)
N_files = len(file_names)

In [4]:
hists_read = []

for iFile in range(N_files):
    infile = root.TFile.Open(file_names[iFile], "read")
    hists_emc_read0 = []
    for itype in range(N_type):
        hist_select_3D0 = []
        for ihist in range(len(hists_all[itype])):
            if iFile < 5: hist_select_3D0.append(infile.Get(hists_all[itype][ihist]))
            hist_select_3D0[-1].SetDirectory(root.nullptr)
            hist_select_3D0[-1].SetName(hist_select_3D0[-1].GetName()+f"{iFile}")
        hists_emc_read0.append(hist_select_3D0)
    hists_read.append(hists_emc_read0)

infile.Close()

In [5]:
ipt = 0
type_loc = 0 
N_dca_files = 2
N_type_loc= len(hists_all[type_loc])
c0 = root.TCanvas(f"c0",f"c0",1400,450*N_dca_files)
c0.Divide(N_type_loc,N_dca_files)
for itype in range(N_type_loc):
    for iFile in range(0,N_dca_files):
        c0.cd(iFile*N_type_loc+itype+1)
        ibin0 = hists_read[iFile][type_loc][itype].ProjectionZ().FindBin(pt_boarders[2*ipt+0])
        ibin1 = hists_read[iFile][type_loc][itype].ProjectionZ().FindBin(pt_boarders[2*ipt+1])
        #hists_read[iFile][type_loc][itype].GetZaxis().SetRange(ibin0,ibin1)
        project= hists_read[iFile][type_loc][itype].ProjectionX(hists_read[iFile][type_loc][itype].GetName()+f"_X_{itype}",1,-1,ibin0,ibin1)
        Format_Hist_total(project, "N_{true}/N^{all}_{assoc.}", "prob. dens.", 0.15, 0.15, 0.01, 0.01,  .07,  0.06, 21,  2, itype+1,  3,  itype+1,  0.95, 1.08, "",  1,  1-0.15*iFile)
        project.SetMinimum(1)
        project.Draw("colz")
        if iFile == 0 :
            project.Draw("H")
            max = project.GetMaximum()
        else:
            project.Scale(max/project.GetMaximum()) 
            project.SetMinimum(1)         
            project.Draw("H same")

c0.Draw()
c0.SaveAs(f"output/rich_emcal_qa/qa_pt_centr_{ipt}.png")

python ERROR: cannot open image file "output/rich_emcal_qa/qa_pt_centr_0.png" for writing. Please check permissions.


In [6]:
type_loc =1
N_type_loc= len(hists_all[type_loc])
c0 = root.TCanvas(f"c1",f"c1",700*2,450*2)
c0.Divide(2,2)
N_dca_files = 2
legends = []
for itype in range(N_type_loc):
    for iselect in range(4):
        max = 0
        legends.append(root.TLegend(0.51,0.6,0.85,0.98))
        Format_Legend(legends[-1],0.03)
        for iFile in range(0,N_dca_files):
            c0.cd(iselect+1)
            ibinY0 = hists_read[iFile][type_loc][itype].ProjectionY().FindBin(pt_boarders[2*(iselect%2)+0])
            ibinY1 = hists_read[iFile][type_loc][itype].ProjectionY().FindBin(pt_boarders[2*(iselect%2)+1])
            ibinZ0 = hists_read[iFile][type_loc][itype].ProjectionZ().FindBin(central_bins[2*int(iselect/2)+0])
            ibinZ1 = hists_read[iFile][type_loc][itype].ProjectionZ().FindBin(central_bins[2*int(iselect/2)+1])
            project = hists_read[iFile][type_loc][itype].ProjectionX(hists_read[iFile][type_loc][itype].GetName()+f"_X_{iselect}",ibinY0,ibinY1,ibinZ0,ibinZ1)
            project.Scale(1./project.GetBinContent(1))
            #project.SetMaximum(100)
            Unity_Line(0,10)
            #for ibin in range(project.GetNbinsX()):
            #    if ibin <=4 : project.GetXaxis().SetBinLabel(ibin,f"{ibin-1}/3")
            #    else :       project.GetXaxis().SetBinLabel(ibin,f"{ibin-1-4}/4")
            Format_Hist_total(project, "N_{true}/N^{all}_{assoc.}", "prob. dens.", 0.15, 0.15, 0.01, 0.01,  .07,  0.06, 21,  2, iFile+1,  7,  iFile+1,  0.95, 1.08, "",  1,  1-0.15*iFile)
            #project.GetXaxis().SetRange(project.FindBin(0),project.FindBin(9))
            #root.gPad.SetLogy()
            if iFile == 0 :
                project.Draw("H")
                max = project.GetMaximum()
            else:
                project.Scale(max/project.GetMaximum())          
                project.Draw("H same")
            legends[-1].AddEntry(project,part_names[iFile],"l")
        legends[-1].Draw()
        legends.append(root.TLegend(0.21,0.78,0.45,0.98))
        Format_Legend(legends[-1],0.09)
        legends[-1].SetHeader("#splitline{"+f"{pt_boarders[2*(iselect%2)+0]}<pT<{pt_boarders[2*(iselect%2)+1]}"+"}{"+f"{central_bins[2*int(iselect/2)+0]}%-{central_bins[2*int(iselect/2)+1]}%"+"}")
        legends[-1].Draw()


c0.Draw()
c0.SaveAs(f"output/rich_emcal_qa/ghost_{ipt}.png")

python ERROR: cannot open image file "output/rich_emcal_qa/ghost_0.png" for writing. Please check permissions.


In [7]:
ipt = 0
type_loc = 0 
N_dca_files = 2
N_type_loc= len(hists_all[type_loc])
c0 = root.TCanvas(f"c0",f"c0",1400,450*N_dca_files)
c0.Divide(N_type_loc,N_dca_files)
for itype in range(N_type_loc):
    for iFile in range(2,N_dca_files+2):
        c0.cd((iFile-2)*N_type_loc+itype+1)
        ibin0 = hists_read[iFile][type_loc][itype].ProjectionZ().FindBin(pt_boarders[2*ipt+0])
        ibin1 = hists_read[iFile][type_loc][itype].ProjectionZ().FindBin(pt_boarders[2*ipt+1])
        #hists_read[iFile][type_loc][itype].GetZaxis().SetRange(ibin0,ibin1)
        project= hists_read[iFile][type_loc][itype].ProjectionX(hists_read[iFile][type_loc][itype].GetName()+f"_X_{itype}",1,-1,ibin0,ibin1)
        Format_Hist_total(project, "N_{true}/N^{all}_{assoc.}", "prob. dens.", 0.15, 0.15, 0.01, 0.01,  .07,  0.06, 21,  2, itype+1,  3,  itype+1,  0.95, 1.08, "",  1,  1-0.15*iFile)
        project.SetMinimum(1)         
        project.Draw("colz")
        if iFile == 2 :
            project.Draw("H")
            max = project.GetMaximum()
        else:
            project.Scale(max/project.GetMaximum())  
            project.SetMinimum(1)                 
            project.Draw("H same")

c0.Draw()
c0.SaveAs(f"output/rich_emcal_qa/qa_pt_centr_{ipt}.png")

python ERROR: cannot open image file "output/rich_emcal_qa/qa_pt_centr_0.png" for writing. Please check permissions.


In [8]:
type_loc =1
N_type_loc= len(hists_all[type_loc])
c0 = root.TCanvas(f"c1",f"c1",700*2,450*2)
c0.Divide(2,2)
N_dca_files = 2
legends = []
for itype in range(N_type_loc):
    for iselect in range(4):
        max = 0
        legends.append(root.TLegend(0.41,0.6,0.75,0.98))
        Format_Legend(legends[-1],0.05)
        for iFile in range(0,N_dca_files):
            c0.cd(iselect+1)
            iFile = iFile+2
            ibinY0 = hists_read[iFile][type_loc][itype].ProjectionY().FindBin(pt_boarders[2*(iselect%2)+0])
            ibinY1 = hists_read[iFile][type_loc][itype].ProjectionY().FindBin(pt_boarders[2*(iselect%2)+1])
            ibinZ0 = hists_read[iFile][type_loc][itype].ProjectionZ().FindBin(central_bins[2*int(iselect/2)+0])
            ibinZ1 = hists_read[iFile][type_loc][itype].ProjectionZ().FindBin(central_bins[2*int(iselect/2)+1])
            project = hists_read[iFile][type_loc][itype].ProjectionX(hists_read[iFile][type_loc][itype].GetName()+f"_X_{iselect}",ibinY0,ibinY1,ibinZ0,ibinZ1)
            project.Scale(1./project.GetBinContent(1))
            #project.SetMaximum(100)
            Unity_Line(0,10)
            #for ibin in range(project.GetNbinsX()):
            #    if ibin <=4 : project.GetXaxis().SetBinLabel(ibin,f"{ibin-1}/3")
            #    else :       project.GetXaxis().SetBinLabel(ibin,f"{ibin-1-4}/4")
            Format_Hist_total(project, "N_{true}/N^{all}_{assoc.}", "prob. dens.", 0.15, 0.15, 0.01, 0.01,  .07,  0.06, 21,  2, iFile+1,  7,  iFile+1,  0.95, 1.08, "",  1,  1-0.15*iFile)
            #project.GetXaxis().SetRange(project.FindBin(0),project.FindBin(9))
            #root.gPad.SetLogy()
            if iFile == 2 :
                project.Draw("H")
                max = project.GetMaximum()
            else:
                project.Scale(max/project.GetMaximum())          
                project.Draw("H same")
            legends[-1].AddEntry(project,part_names[iFile],"l")
        legends[-1].Draw()
        legends.append(root.TLegend(0.21,0.78,0.45,0.98))
        Format_Legend(legends[-1],0.09)
        legends[-1].SetHeader("#splitline{"+f"{pt_boarders[2*(iselect%2)+0]}<pT<{pt_boarders[2*(iselect%2)+1]}"+"}{"+f"{central_bins[2*int(iselect/2)+0]}%-{central_bins[2*int(iselect/2)+1]}%"+"}")
        legends[-1].Draw()


c0.Draw()
c0.SaveAs(f"output/rich_emcal_qa/ghost_{ipt}.png")

python ERROR: cannot open image file "output/rich_emcal_qa/ghost_0.png" for writing. Please check permissions.
